## Homework 2

Instructions
Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).

Складові генерації: АЕС,ТЕЦ,ВДЕ,ТЕС,ГЕС,ГАЕС. Споживання не включає обсяги закачки ГАЕС. Міждержавні перетоки: Україна - Білорусь та РФ, Україна - ЄС, Україна - Молдова. Одиниця виміру - МВт.

In [393]:
import   altair as alt
import   pandas as pd
import   datetime

alt.data_transformers.enable('csv')
alt.renderers.enable('default')

RendererRegistry.enable('default')

### 1. Data investigation

In [394]:
df = pd.read_excel('./2014-2020.xlsx')
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [395]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61368 entries, 0 to 61367
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Час/Дата     61368 non-null  object 
 1   AES          61368 non-null  int64  
 2   TEC          61368 non-null  int64  
 3   VDE          61368 non-null  int64  
 4   TES          61368 non-null  int64  
 5   GES          61367 non-null  float64
 6   GAES_GEN     61368 non-null  int64  
 7   CONSUMPTION  61368 non-null  int64  
 8   GAES_PUMP    61367 non-null  float64
 9   UK_BLR_RUS   61367 non-null  float64
 10  UK_EURO      61367 non-null  float64
 11  UK_MLD       61367 non-null  float64
 12  Unnamed: 12  1 non-null      float64
dtypes: float64(6), int64(6), object(1)
memory usage: 6.1+ MB


### 2. Data preprocessing

In [396]:
# Delete strange column and ones we won't use

df.drop("Unnamed: 12",  axis='columns', inplace=True)
df.drop("GAES_PUMP",    axis='columns', inplace=True)
df.drop("UK_BLR_RUS",   axis='columns', inplace=True)
df.drop("UK_EURO",      axis='columns', inplace=True)
df.drop("UK_MLD",       axis='columns', inplace=True)

In [397]:
# Find rows where are Nan values

df[df.isnull().any(axis=1)]

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION
17544,24-31.12.2018,10533,2345,11,5882,NaN,0,17883


In [398]:
# Lets drop this row 17544, due to avoid incomplete picture

df.drop(df.index[[17544]], inplace=True)

In [399]:
# Transform columns

df["Hour"]     = df["Час/Дата"].str.split("-",expand=True)[0]
df["Date"]     = df["Час/Дата"].str.split("-",expand=True)[1]
df["Day"]      = df["Date"].str.split(".",expand=True)[0]
df["Month"]    = df["Date"].str.split(".",expand=True)[1]
df["Year"]     = df["Date"].str.split(".",expand=True)[2]
df["Hour"]     = pd.to_numeric(df["Hour"])
df["Day"]      = pd.to_numeric(df["Day"])
df["Month"]    = pd.to_numeric(df["Month"])
df["Year"]     = pd.to_numeric(df["Year"])
df["Date"]     = pd.to_datetime(df["Date"])
df["DayMonth"] = df.apply (lambda row: row["Date"].strftime('%j'), axis=1) 
df["DayMonth"] = pd.to_numeric(df["DayMonth"])
df['Weekday']  = pd.Series(df["Date"]).dt.day_name()

In [400]:
# Add season column for next visualization

season_month = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'
}
df["Season"] = df.apply (lambda row: season_month.get(row["Month"]), axis=1)

In [401]:
# Needed for sorting

hours_sorted   = list(range(1, 25))
days_sorted    = list(range(1, 367))
months_sorted  = list(range(1, 13))
seasons_sorted = ['Winter', 'Spring', 'Summer', 'Autumn']

In [402]:
# Drop not needed column and rename one to GAES

df.drop("Час/Дата", axis='columns', inplace=True)
df = df.rename(columns = {'GAES_GEN': 'GAES'}, inplace = False)

In [403]:
df

,AES,TEC,VDE,TES,GES,GAES,CONSUMPTION,Hour,Date,Day,Month,Year,DayMonth,Weekday,Season
0,9235,2039,621,4942,385.0,0,16693,24,2020-12-31,31,12,2020,366,Thursday,Winter
1,9221,2159,707,5549,470.0,0,17805,23,2020-12-31,31,12,2020,366,Thursday,Winter
2,9249,2377,709,5906,1000.0,0,18870,22,2020-12-31,31,12,2020,366,Thursday,Winter
3,9256,2499,702,6329,909.0,322,19887,21,2020-12-31,31,12,2020,366,Thursday,Winter
4,9213,2521,702,6640,823.0,602,20387,20,2020-12-31,31,12,2020,366,Thursday,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61363,10427,2826,49,5506,274.0,0,18072,5,2014-01-01,1,1,2014,1,Wednesday,Winter
61364,10475,2827,42,5524,185.0,0,18453,4,2014-01-01,1,1,2014,1,Wednesday,Winter
61365,10515,2837,54,5865,355.0,0,19107,3,2014-01-01,1,1,2014,1,Wednesday,Winter
61366,10606,2822,49,6038,863.0,0,19665,2,2014-01-01,1,1,2014,1,Wednesday,Winter


In [404]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61367 entries, 0 to 61367
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   AES          61367 non-null  int64         
 1   TEC          61367 non-null  int64         
 2   VDE          61367 non-null  int64         
 3   TES          61367 non-null  int64         
 4   GES          61367 non-null  float64       
 5   GAES         61367 non-null  int64         
 6   CONSUMPTION  61367 non-null  int64         
 7   Hour         61367 non-null  int64         
 8   Date         61367 non-null  datetime64[ns]
 9   Day          61367 non-null  int64         
 10  Month        61367 non-null  int64         
 11  Year         61367 non-null  int64         
 12  DayMonth     61367 non-null  int64         
 13  Weekday      61367 non-null  object        
 14  Season       61367 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(11), object(2

### 3. Plotting

#### Task 1. Як змінювалась структура генерації електроенергії за роками?

In [405]:
task1_df = df[['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES', 'Year']]
task1_df = task1_df.groupby('Year').sum()
task1_df = task1_df.reset_index()
task1_df

,Year,AES,TEC,VDE,TES,GES,GAES
0,2014,88204418,14684731,1606386,68605877,7991074.0,839842
1,2015,87413763,12041074,1234936,50259819,4964478.0,1554857
2,2016,80762094,12841033,1093190,49879543,7113989.0,1621377
3,2017,85314258,12208062,1177032,44945303,8531952.0,1575099
4,2018,84340795,12258673,1863715,47741210,10072767.0,1564867
5,2019,83098265,12251850,4441801,44877587,6422154.0,1323192
6,2020,76210883,14536545,8124734,39504024,5998680.0,1561564


In [406]:
alt.Chart(task1_df).mark_bar(
    cornerRadiusTopLeft  = 3,
    cornerRadiusTopRight = 3
).transform_fold(
    ['AES', 'TEC', 'VDE', 'TES', 'GES','GAES']
).encode(
    x = alt.X('value:Q', title='Кількість згенерованої електроенергії (МВт)'),
    y = alt.Y('Year:N', title="Рік"),
    color = alt.Color('key:N', title='Складові'),
    order=alt.Order(
      'key:N',
       sort='ascending'
    )
).properties(width = 800, height = 400, title='Як змінювалась структура генерації електроенергії за роками?')

alt.Chart(...)

##### Task 1: було обрано звичайний bar chart, адже у нас тут є потреба показати тільки кількість генерації кожної складової, по відношенню до року. Bar chart досить добре справляється з таким, задля кращої зорової читабельності зробив його горизонтальним. 

#### Task 2. Як залежить споживання електроенергії від дня року та години доби?

In [407]:
task2_df = df[['DayMonth', 'Hour', 'CONSUMPTION']]
task2_df = task2_df.groupby(["DayMonth", "Hour"]).mean()
task2_df = task2_df.reset_index()
task2_df

,DayMonth,Hour,CONSUMPTION
0,1,1,18202.428571
1,1,2,17436.142857
2,1,3,16873.428571
3,1,4,16460.571429
4,1,5,16287.142857
...,...,...,...
8779,366,20,20740.000000
8780,366,21,20227.500000
8781,366,22,19492.000000
8782,366,23,18561.000000


In [408]:
alt.Chart(task2_df).mark_rect().encode(
    x = alt.X('DayMonth:O', title='День року', sort=days_sorted),
    y = alt.Y('Hour:O', title='Година', sort=hours_sorted),
    color = alt.Color('CONSUMPTION:Q', title='Спожито - середнє значення', scale=alt.Scale(scheme="inferno"))
).properties(width = 600, 
             height = 600, 
             title='Як залежить споживання електроенергії від дня року та години доби у середньому за 2014-2020рр.?')

alt.Chart(...)

##### Task 2: тут вже звичайним bar chart не обійдешся, адже нам потрібно показати кількість спожитої електроенергії по відношенню до двох інших параметрів - години і дня року. Heatmap для цього на мою думку підходить найкраще, адже легко побачити що зимові дні у році потребують більшого споживання.

#### Task 3. Як змінюється генерація електроенергії з різних джерел впродовж доби?

In [409]:
task3_df = df[['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES','Hour']]
task3_df = task3_df.groupby('Hour').mean()
task3_df = task3_df.reset_index()
task3_df.head()

,Hour,AES,TEC,VDE,TES,GES,GAES
0,1,9533.777865,1428.560813,164.178334,4883.335549,413.657020,0.26711
1,2,9530.698084,1424.868987,164.754009,4772.738365,325.804849,0.00000
2,3,9530.246774,1425.223309,164.704341,4722.442315,314.622214,0.00000
3,4,9531.818537,1423.511928,164.179898,4684.545561,289.973797,0.00000
4,5,9535.267110,1425.863512,165.059836,4697.965976,323.253813,0.00000


In [410]:
alt.Chart(task3_df).mark_bar(
    cornerRadiusTopLeft  = 3,
    cornerRadiusTopRight = 3
).transform_fold(
    ['AES', 'TEC', 'VDE', 'TES', 'GES','GAES']
).encode(
    x = alt.X('value:Q', title='Кількість згенерованої електроенергії (МВт) - середнє значення за 2014-2020рр.'),
    y = alt.Y('Hour:N', title="Година", sort=hours_sorted),
    color = alt.Color('key:N', title='Складові'),
    order=alt.Order(
      'key:N',
       sort='ascending'
    )
).properties(width = 800, height = 400, title='Як змінюється генерація електроенергії з різних джерел впродовж доби?')

alt.Chart(...)

##### Task 3: було обрано звичайний bar chart, адже у нас тут є потреба показати тільки кількість генерації кожної складової, по відношенню до години. Bar chart досить добре справляється з таким, задля кращої зорової читабельності зробив його горизонтальним. 

#### Task 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

In [411]:
task4_1_df = df[['Hour', 'Month','CONSUMPTION']]
task4_1_df = task4_1_df.groupby(["Hour", "Month"]).mean()
task4_1_df = task4_1_df.reset_index()
task4_1_df

,Hour,Month,CONSUMPTION
0,1,1,17855.262673
1,1,2,17505.383838
2,1,3,16270.645161
3,1,4,14712.419048
4,1,5,13518.317972
...,...,...,...
283,24,8,14306.423963
284,24,9,14024.500000
285,24,10,15610.626728
286,24,11,17193.147619


In [412]:
alt.Chart(task4_1_df).mark_rect().encode(
    x = alt.X('Hour:O', title='Година', sort=hours_sorted),
    y = alt.Y('Month:O', title='Місяць', sort=months_sorted),
    color = alt.Color('CONSUMPTION:Q', title='Спожито - середнє значення', scale=alt.Scale(scheme="inferno"))
).properties(width = 600, 
             height = 600, 
             title='Як змінюється споживання електроенергії впродовж доби у розрізі місяців року за 2014-2020рр?')

alt.Chart(...)

In [413]:
task4_2_df = df[['Hour','Season','CONSUMPTION']]
task4_2_df = task4_2_df.groupby(["Hour", "Season"]).mean()
task4_2_df = task4_2_df.reset_index()
task4_2_df

,Hour,Season,CONSUMPTION
0,1,Autumn,14731.065934
1,1,Spring,14835.113354
2,1,Summer,13697.197205
3,1,Winter,17547.101266
4,2,Autumn,14262.481947
...,...,...,...
91,23,Winter,19331.330696
92,24,Autumn,15609.437991
93,24,Spring,15644.263975
94,24,Summer,14515.498447


In [414]:
alt.Chart(task4_2_df).mark_rect().encode(
    x = alt.X('Hour:O', title='Година', sort=hours_sorted),
    y = alt.Y('Season:O', title='Пора року', sort=seasons_sorted),
    color = alt.Color('CONSUMPTION:Q', title='Спожито - середнє значення', scale=alt.Scale(scheme="inferno"))
).properties(width = 600, 
             height = 600, 
             title='Як змінюється споживання електроенергії впродовж доби у розрізі пір року за 2014-2020рр?')

alt.Chart(...)

##### Task 4: тут вже звичайним bar chart не обійдешся, адже нам потрібно показати кількість спожитої електроенергії по відношенню до двох інших параметрів - години і місяця/пори року. Heatmap для цього на мою думку підходить найкраще, адже легко побачити що зимові дні у році потребують більшого споживання.

#### Task 5. Як змінюється споживання електроенергії впродовж тижня?

In [415]:
task5_df = df[['CONSUMPTION','Weekday']]
task5_df = task5_df.groupby('Weekday').mean()
task5_df = task5_df.reset_index()
task5_df

,Weekday,CONSUMPTION
0,Friday,17222.637785
1,Monday,17116.894509
2,Saturday,16814.657420
3,Sunday,16544.002626
4,Thursday,17293.868511
5,Tuesday,17233.914269
6,Wednesday,17276.704007


In [416]:
alt.Chart(task5_df).mark_bar(
    cornerRadiusTopLeft  = 3,
    cornerRadiusTopRight = 3
).encode(
    x = alt.X('CONSUMPTION:Q', 
              scale=alt.Scale(domain=[16500, 17400]), 
              title='Кількість спожитої електроенергії (МВт) - середнє значення за 2014-2020рр.'
             ),
    y = alt.Y('Weekday:O',     title="День тижня",   sort='-x'),
    color=alt.condition(
        alt.datum.Weekday == 'Thursday',
        alt.value('green'),
        alt.value('lightgrey')
     )
).properties(width = 600, height = 300, title='Як змінюється споживання електроенергії впродовж тижня?')

alt.Chart(...)

##### Task 5: було обрано звичайний bar chart, адже у нас тут є потреба показати тільки середнє значення споживання по відношенню до дня тижня. Bar chart досить добре справляється з таким, задля кращої зорової читабельності зробив його горизонтальним. Також я виокремив окремим кольором день тижня, в який відбувається найбільше споживання. 